# Tutorial

The first step is to install stratopy. To install it, on a terminal run

```console
$ pip install stratopy
```

After installing, the first thing to do is to load the stratopy package. 

## The stratopy package

We always recommend to read the documentation.

In [47]:
import stratopy
print(stratopy.__doc__)

A Python package designed to easily manipulate CloudSat and GOES-R
and generate labeled images containing cloud types.

It consists in five modules:

- stratopy.cloudsat module
- stratopy.core module
- stratopy.goes module
- stratopy.IO module
- stratopy.merger module
 


This tutorial will guide you through all five modules.

## Using the IO Module

Stratopy is intended to be used by data scientists that are interested in trainning machine learning algorithms based on satellite imagery data. `Stratopy` grants the possibility of downloading images from remote servers via the IO module. 

In case of having access to CloudSat's remote server, then by defining your username and password as strings and by passing the route to the desired image it is possible to download it (insert your username and password): 

In [51]:
from stratopy import IO

username = ''
password = ''
route = '2B-CLDCLASS.P1_R05/2019/003/2019003151948_67564_CS_2B-CLDCLASS_GRANULE_P1_R05_E08_F03.hdf'

IO.fetch_cloudsat(dirname = route,
                 user = username,
                 passwd = password,
                 host = 'ftp.cloudsat.cira.colostate.edu',
                 tag = 'stratopy-cloudsat')

,read_time,Longitude,Latitude,layer_0,layer_1,layer_2,layer_3,layer_4,layer_5,layer_6,layer_7,layer_8,layer_9
0,2019-01-03 15:19:58.450000000,156.896011,-0.006094,0,0,0,0,0,0,0,0,0,0
1,2019-01-03 15:19:58.609999967,156.893967,-0.015801,0,0,0,0,0,0,0,0,0,0
2,2019-01-03 15:19:58.769999933,156.891907,-0.025507,0,0,0,0,0,0,0,0,0,0
3,2019-01-03 15:19:58.929999900,156.889862,-0.035213,0,0,0,0,0,0,0,0,0,0
4,2019-01-03 15:19:59.089999986,156.887802,-0.044920,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36945,2019-01-03 16:58:29.651171875,132.272720,0.043348,0,0,0,0,0,0,0,0,0,0
36946,2019-01-03 16:58:29.810839844,132.270676,0.033642,0,0,0,0,0,0,0,0,0,0
36947,2019-01-03 16:58:29.970996094,132.268616,0.023935,0,0,0,0,0,0,0,0,0,0
36948,2019-01-03 16:58:30.131152344,132.266571,0.014229,0,0,0,0,0,0,0,0,0,0


## Using the merger Module

In [1]:
from stratopy import merger
print(merger.__doc__)

Contains the principal Data Frame class which allows for merging
satellite images.


## About the cloudsat Module

In [46]:
from stratopy import cloudsat
print(cloudsat.__doc__)

Module containing all CloudSat related classes and methods.


## About the goes Module

In [45]:
from stratopy import goes

print(goes.__doc__)

Module containing all GOES satellite related classes and methods.


## Understanding the core Module

In [43]:
from stratopy import core

print(core.__doc__)

Contains methods to perform transformation operations \
on loaded images.
